<CENTER>
    <a href="http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html" class="icons"><img src="../../images/ATLASOD.gif" style="width:40%"></a>
</CENTER>

<CENTER><h1>Searching for the Higgs boson in the $pp \rightarrow t\bar{t}$ channel</h1></CENTER>

## C++ notebook example

**Introduction**
Let's take a current ATLAS Open Data sample and create a histogram:

We need to include some standard C++ and ROOT libraries

In [1]:
// Creates a TChain to be used by the Analysis.C class
#include <TChain.h>
#include <vector>
#include <TFile.h>
#include <iostream>
#include <string>
#include <stdio.h>

Because we would like to use more than one ROOT input file, the best option is to use a TChain object. This allows to "chain" several samples into a single structure that we can later loop over

In [2]:
TString path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/1lep/";

In [3]:
TChain* fChain = new TChain("mini");

fChain->AddFile(path+"Data/data_A.1lep.root");
fChain->AddFile(path+"Data/data_B.1lep.root");
fChain->AddFile(path+"Data/data_C.1lep.root");
fChain->AddFile(path+"Data/data_D.1lep.root");

Now we're going to extract the lepton and jet variables

In [4]:
UInt_t                lep_n;
vector<float>         *lep_truthMatched = 0;
vector<float>         *lep_trigMatched = 0;
vector<float>         *lep_pt = 0;
vector<float>         *lep_eta = 0;
vector<float>         *lep_phi = 0;
vector<float>         *lep_E = 0;
vector<float>         *lep_z0 = 0;
vector<int>           *lep_charge = 0;
vector<unsigned int>  *lep_type = 0;
vector<bool>          *lep_isTightID = 0;
vector<float>         *lep_ptcone30 = 0;
vector<float>         *lep_etcone20 = 0;
vector<float>         *lep_trackd0pvunbiased = 0;
vector<float>         *lep_tracksigd0pvunbiased = 0;
vector<float>         *lep_pt_syst = 0;

UInt_t                jet_n;
vector<float>         *jet_pt = 0;
vector<float>         *jet_eta = 0;
vector<float>         *jet_phi = 0;
vector<float>         *jet_E = 0;
vector<float>         *jet_jvt = 0;
vector<int>           *jet_trueflav = 0;
vector<bool>          *jet_truthMatched = 0;
vector<float>         *jet_MV2c10 = 0;
vector<float>         *jet_pt_syst = 0;

Float_t               met_et;
Float_t               met_phi;
Float_t               met_et_syst;

Let's declare some other useful variables:

In [5]:
Int_t           runNumber;
Int_t           eventNumber;
Int_t           channelNumber;
Float_t         mcWeight;
Float_t         scaleFactor_PILEUP;
Float_t         scaleFactor_ELE;
Float_t         scaleFactor_MUON;
Float_t         scaleFactor_PHOTON;
Float_t         scaleFactor_TAU;
Float_t         scaleFactor_BTAG;
Float_t         scaleFactor_LepTRIGGER;
Float_t         scaleFactor_PhotonTRIGGER;
Float_t         scaleFactor_TauTRIGGER;
Float_t         scaleFactor_DiTauTRIGGER;
Bool_t          trigE;
Bool_t          trigM;
Bool_t          trigP;
Bool_t          trigT;
Bool_t          trigDT;

Let's declare some branches.

In [6]:
TBranch        *b_runNumber;   
TBranch        *b_eventNumber;   
TBranch        *b_channelNumber;   
TBranch        *b_mcWeight;   
TBranch        *b_scaleFactor_PILEUP;   
TBranch        *b_scaleFactor_ELE;   
TBranch        *b_scaleFactor_MUON; 
TBranch        *b_scaleFactor_PHOTON;   
TBranch        *b_scaleFactor_TAU;   
TBranch        *b_scaleFactor_BTAG;   
TBranch        *b_scaleFactor_LepTRIGGER;   
TBranch        *b_scaleFactor_PhotonTRIGGER;   
TBranch        *b_scaleFactor_TauTRIGGER;   
TBranch        *b_scaleFactor_DiTauTRIGGER;   
TBranch        *b_trigE;   
TBranch        *b_trigM;   
TBranch        *b_trigP;   
TBranch        *b_trigT;   
TBranch        *b_trigDT;   

TBranch        *b_lep_n;  
TBranch        *b_lep_truthMatched;  
TBranch        *b_lep_trigMatched;  
TBranch        *b_lep_pt;  
TBranch        *b_lep_eta;   
TBranch        *b_lep_phi;  
TBranch        *b_lep_E;  
TBranch        *b_lep_z0;  
TBranch        *b_lep_charge;   
TBranch        *b_lep_type;   
TBranch        *b_lep_isTightID;   
TBranch        *b_lep_ptcone30;   
TBranch        *b_lep_etcone20;   
TBranch        *b_lep_trackd0pvunbiased;   
TBranch        *b_lep_tracksigd0pvunbiased;   
TBranch        *b_met_et;   
TBranch        *b_met_phi;   
TBranch        *b_jet_n;   
TBranch        *b_jet_pt;   
TBranch        *b_jet_eta;  
TBranch        *b_jet_phi;  
TBranch        *b_jet_E;   
TBranch        *b_jet_jvt; 
TBranch        *b_jet_trueflav;   
TBranch        *b_jet_truthMatched;   
TBranch        *b_jet_MV2c10;   
TBranch        *b_lep_pt_syst;   
TBranch        *b_met_et_syst;   
TBranch        *b_jet_pt_syst;   

Here we're filling the variables defined above with the content of those inside the input ntuples

In [7]:
fChain->SetBranchAddress("lep_n", &lep_n, &b_lep_n);
fChain->SetBranchAddress("lep_truthMatched", &lep_truthMatched, &b_lep_truthMatched);
fChain->SetBranchAddress("lep_trigMatched", &lep_trigMatched, &b_lep_trigMatched);
fChain->SetBranchAddress("lep_pt", &lep_pt, &b_lep_pt);
fChain->SetBranchAddress("lep_eta", &lep_eta, &b_lep_eta);
fChain->SetBranchAddress("lep_phi", &lep_phi, &b_lep_phi);
fChain->SetBranchAddress("lep_E", &lep_E, &b_lep_E);
fChain->SetBranchAddress("lep_z0", &lep_z0, &b_lep_z0);
fChain->SetBranchAddress("lep_charge", &lep_charge, &b_lep_charge);
fChain->SetBranchAddress("lep_type", &lep_type, &b_lep_type);
fChain->SetBranchAddress("lep_isTightID", &lep_isTightID, &b_lep_isTightID);
fChain->SetBranchAddress("lep_ptcone30", &lep_ptcone30, &b_lep_ptcone30);
fChain->SetBranchAddress("lep_etcone20", &lep_etcone20, &b_lep_etcone20);
fChain->SetBranchAddress("lep_trackd0pvunbiased", &lep_trackd0pvunbiased, &b_lep_trackd0pvunbiased);
fChain->SetBranchAddress("lep_tracksigd0pvunbiased", &lep_tracksigd0pvunbiased, &b_lep_tracksigd0pvunbiased);
fChain->SetBranchAddress("met_et", &met_et, &b_met_et);
fChain->SetBranchAddress("met_phi", &met_phi, &b_met_phi);
fChain->SetBranchAddress("jet_n", &jet_n, &b_jet_n);
fChain->SetBranchAddress("jet_pt", &jet_pt, &b_jet_pt);
fChain->SetBranchAddress("jet_eta", &jet_eta, &b_jet_eta);
fChain->SetBranchAddress("jet_phi", &jet_phi, &b_jet_phi);
fChain->SetBranchAddress("jet_E", &jet_E, &b_jet_E);
fChain->SetBranchAddress("jet_jvt", &jet_jvt, &b_jet_jvt);
fChain->SetBranchAddress("jet_trueflav", &jet_trueflav, &b_jet_trueflav);
fChain->SetBranchAddress("jet_truthMatched", &jet_truthMatched, &b_jet_truthMatched);
fChain->SetBranchAddress("jet_MV2c10", &jet_MV2c10, &b_jet_MV2c10);
fChain->SetBranchAddress("lep_pt_syst", &lep_pt_syst, &b_lep_pt_syst);
fChain->SetBranchAddress("met_et_syst", &met_et_syst, &b_met_et_syst);
fChain->SetBranchAddress("jet_pt_syst", &jet_pt_syst, &b_jet_pt_syst);

We're creating a histogram for this example. The plan in to fill them with events.

In [8]:
// Global histograms
hist_etmiss       = new TH1F("hist_etmiss",      "Missing Transverse Momentum;E_{T}^{miss} [GeV];Events / bin", 20, 0, 300);
hist_mtw          = new TH1F("hist_mtw",         "Transverse Mass; M^{W}_{T} [GeV];Events / bin", 20, 0, 300);
hist_syst_etmiss  = new TH1F("hist_syst_etmiss",      "Missing Transverse Momentum (MET syst);E_{T}^{miss} [GeV] (after E_{T}^{miss} syst. uncert.);Events / bin", 20, 0, 300);
hist_syst_mtw     = new TH1F("hist_syst_mtw",         "Transverse Mass (MET syst); M^{W}_{T} [GeV] (after E_{T}^{miss} and p_{T}^{lep} syst. uncert.);Events / bin", 20, 0, 300);

// Jet variables histograms
hist_n_jets           = new TH1F("hist_n_jets",          "Number of Jets;N_{jets};Events", 5, 3.5, 8.5);
hist_leadjet_pt       = new TH1F("hist_leadjet_pt",      "Leading Jet Transverse Momentum;p_{T}^{jet} [GeV];Events / bin", 20, 0, 450);
hist_syst_leadjet_pt  = new TH1F("hist_syst_leadjet_pt",      "Leading Jet Transverse Momentum;p_{T}^{jet} [GeV] (after p_{T}^{jet} syst. uncert.);Events / bin", 20, 0, 450);
hist_leadjet_eta      = new TH1F("hist_leadjet_eta",     "Leading Jet Pseudorapidity; #eta^{jet}; Events / bin", 30, -3, 3);
hist_n_bjets          = new TH1F("hist_n_bjets",         "Number of b-jets;N_{b-jets};Events", 3, 1.5, 4.5);
hist_leadbjet_pt      = new TH1F("hist_leadbjet_pt",     "Leading b-jet Transverse Momentum;p_{T}^{b-jet} [GeV];Events / bin", 20, 0, 350);
hist_leadbjet_eta     = new TH1F("hist_leadbjet_eta",    "Leading b-jet Pseudorapidity; #eta^{b-jet}; Events / bin", 30, -3, 3);

// W and top variables
hist_Topmass       = new TH1F("hist_Topmass",      "Mass of three jets;m_{jjj}^{max p_{T}} [GeV];Events / bin", 40, 100, 250);
hist_Wmass       = new TH1F("hist_Wmass",      "Mass of two jets;m_{jj}^{max p_{T}} [GeV];Events / bin", 40, 50, 120);

// Leading Lepton histograms
hist_leadleptpt   = new TH1F("hist_leadleptpt",  "Leading Lepton Transverse Momentum;p_{T}^{leadlep} [GeV];Events / bin", 20, 0, 200);
hist_syst_leadleptpt  = new TH1F("hist_syst_leadleptpt",  "Leading Lepton Transverse Momentum;p_{T}^{leadlep} [GeV] (after p_{T}^{lep} syst. uncert.);Events / bin", 20, 0, 200);
 
hist_leadlepteta  = new TH1F("hist_leadlepteta", "Leading Lepton Pseudorapidity; #eta^{leadlep}; Events / bin", 30, -3, 3);
hist_leadleptE    = new TH1F("hist_leadleptE",   "Leading Lepton Energy; E^{leadlep} [GeV]; Events / bin", 30, 0, 300);
hist_leadleptphi  = new TH1F("hist_leadleptphi", "Leading Lepton Azimuthal Angle ; #phi^{leadlep}; Events / bin", 32, -3.2, 3.2);
hist_leadleptch   = new TH1F("hist_leadleptch",  "Leading Lepton Charge; Q^{leadlep}; Events / bin", 7, -1.75, 1.75);
hist_leadleptID   = new TH1F("hist_leadleptID",  "Leading Lepton Absolute PDG ID; |PDG ID|^{leadlep}; Events / bin",  15, 5.5, 20.5);
hist_leadlept_ptc  = new TH1F("hist_leadlept_ptc", "Leading Lepton Relative Transverse Momentum Isolation; ptconerel30^{leadlep}; Events / bin", 20, -0.1, 0.2);
hist_leadleptetc  = new TH1F("hist_leadleptetc", "Leading Lepton Relative Transverse Energy Isolation; etconerel20^{leadlep}; Events / bin", 20, -0.1, 0.2);
hist_leadlepz0    = new TH1F("hist_leadlepz0",   "Leading Lepton z0 impact parameter; z_{0}^{leadlep} [mm]; Events / bin", 20, -1, 1);
hist_leadlepd0    = new TH1F("hist_leadlepd0",   "Leading Lepton d0 impact parameter; d_{0}^{leadlep} [mm]; Events / bin", 20, -0.2, 0.2);

Let's define some histograms for the histograms filling:

In [10]:
void FillHistogramsLeadlept( double m, float w , TString s)
{
  //Leading lepton histograms
  if (s.Contains("hist_leadleptpt")) hist_leadleptpt->Fill(m,w);
  if (s.Contains("hist_syst_leadleptpt")) hist_syst_leadleptpt->Fill(m,w);
  if (s.Contains("hist_leadlepteta")) hist_leadlepteta->Fill(m,w);
  if (s.Contains("hist_leadleptE")) hist_leadleptE->Fill(m,w);
  if (s.Contains("hist_leadleptphi")) hist_leadleptphi->Fill(m,w);
  if (s.Contains("hist_leadleptch")) hist_leadleptch->Fill(m,w);
  if (s.Contains("hist_leadleptID")) hist_leadleptID->Fill(m,w);
  if (s.Contains("hist_leadlept_ptc")) hist_leadlept_ptc->Fill(m,w);
  if (s.Contains("hist_leadleptetc")) hist_leadleptetc->Fill(m,w);
  if (s.Contains("hist_leadlepz0")) hist_leadlepz0->Fill(m,w);
  if (s.Contains("hist_leadlepd0")) hist_leadlepd0->Fill(m,w);
}

In [11]:
void FillHistogramsLeadJet( double m, float w , TString s)
{
  //Leading Jet histograms
  if (s.Contains("hist_n_jets")) hist_n_jets->Fill(m,w);
  if (s.Contains("hist_leadjet_pt")) hist_leadjet_pt->Fill(m,w);
  if (s.Contains("hist_syst_leadjet_pt")) hist_syst_leadjet_pt->Fill(m,w);
  if (s.Contains("hist_leadjet_eta")) hist_leadjet_eta->Fill(m,w);
  if (s.Contains("hist_n_bjets")) hist_n_bjets->Fill(m,w);
  if (s.Contains("hist_leadbjet_pt")) hist_leadbjet_pt->Fill(m,w);
  if (s.Contains("hist_leadbjet_eta")) hist_leadbjet_eta->Fill(m,w);
}

In [12]:
void FillHistogramsGlobal( double m, float w , TString s)
{
  //Fill Global histograms
  if (s.Contains("hist_etmiss")) hist_etmiss->Fill(m,w);
  if (s.Contains("hist_mtw")) hist_mtw->Fill(m,w);
  if (s.Contains("hist_syst_etmiss")) hist_syst_etmiss->Fill(m,w);
  if (s.Contains("hist_syst_mtw")) hist_syst_mtw->Fill(m,w);
}

In [13]:
void FillHistogramsTTbar( double m, float w , TString s)
{
  if (s.Contains("hist_Topmass")) hist_Topmass->Fill(m,w);
  if (s.Contains("hist_Wmass")) hist_Wmass->Fill(m,w);
}

The top quark analysis implemented here considers top decays into a $t\bar{t}$. The event selection criteria are (this will take a few minutes):

In [14]:
int nentries, nbytes, i;
nentries = (Int_t)fChain->GetEntries();

std::cout << "* Total number of entries to analyse: " << nentries << std::endl;

* Total number of entries to analyse: 168533115


In [ ]:
//Scale factors (adding b-tagging as it is used)
Float_t scaleFactor = scaleFactor_ELE*scaleFactor_MUON*scaleFactor_LepTRIGGER*scaleFactor_PILEUP*scaleFactor_BTAG;

//MC weight
Float_t m_mcWeight = mcWeight;

//Total weight
Float_t weight = scaleFactor*m_mcWeight;


for (i=0; i < nentries; i++)
{
 nbytes =  fChain->GetEntry(i);
    
      // cut on at least 4 jets
      if (jet_n > 3)
    {
      
      // MET > 30 GeV
      if(met_et > 30000.)
        {
      std::cout << "ciao" << endl;
          // Preselection cut for electron/muon trigger, Good Run List, and good vertex
          if(trigE || trigM)
        {
          
          // Preselection of good leptons
          int goodlep_index =0;
          int goodlep_n = 0;
          int lep_index =0;
          
          for(unsigned int i=0; i<lep_n; i++)
            {
                      TLorentzVector leptemp;  leptemp.SetPtEtaPhiE(lep_pt->at(i)/1000., lep_eta->at(i), lep_phi->at(i), lep_E->at(i)/1000.);

              std::cout << "ciao" << endl;
              // Lepton is Tight
              if( lep_isTightID->at(i) )
            {
              // Lepton is isolated and hard pT
              if( lep_pt->at(i) > 30000. && ( (lep_ptcone30->at(i)/lep_pt->at(i)) < 0.15) && ( (lep_etcone20->at(i) / lep_pt->at(i)) < 0.15 ) )
                {
                  // electron selection in fiducial region excluding candidates in the transition region between the barrel and endcap electromagnetic calorimeters
                  if ( lep_type->at(i)==11 && TMath::Abs(lep_eta->at(i)) < 2.47 && ( TMath::Abs(lep_eta->at(i)) < 1.37 || TMath::Abs(lep_eta->at(i)) > 1.52 ) ) {
                if( TMath::Abs(lep_trackd0pvunbiased->at(i))/lep_tracksigd0pvunbiased->at(i) < 5 && TMath::Abs(lep_z0->at(i)*TMath::Sin(leptemp.Theta())) < 0.5) {
                  goodlep_n = goodlep_n + 1;
                  goodlep_index = i;
                  lep_index++;
                }
                  }
                  // muon selection
                  if ( lep_type->at(i) == 13 && TMath::Abs(lep_eta->at(i)) < 2.5 ) {
                if( TMath::Abs(lep_trackd0pvunbiased->at(i))/lep_tracksigd0pvunbiased->at(i) < 3 && TMath::Abs(lep_z0->at(i)*TMath::Sin(leptemp.Theta())) < 0.5) {
  
                  goodlep_n = goodlep_n + 1;
                  goodlep_index = i;
                  lep_index++;
                }
                  }
                }
            }
            }
          
          
          //Exactly one good lepton
          if(goodlep_n==1)
            {
      
      
              //Preselection of good jets
              int goodjet_n = 0;
              int goodbjet_n = 0;
              
              int goodjet_index[jet_n];
              int jet_index = 0;
      
              int goodbjet_index[jet_n];
              int bjet_index = 0;
      
     
      
              for(unsigned int i=0; i<jet_n; i++)
            {
              if(jet_pt->at(i) > 30000. && TMath::Abs(jet_eta->at(i)) < 2.5)
                {
                  // JVT cleaning
                  bool jvt_pass=true;
                  if (jet_pt->at(i) < 60000. && TMath::Abs(jet_eta->at(i)) < 2.4 && jet_jvt->at(i) < 0.59) jvt_pass=false;
                  if (jvt_pass) 
                {
                  goodjet_n++;
                  goodjet_index[jet_index] = i;
                  jet_index++;
                  
                  // cut on 0.8244273 is 70% WP	
                  if (jet_MV2c10->at(i) >0.8244273	 )
                    {
                      goodbjet_n++;
                      goodbjet_index[bjet_index] = i;
                      bjet_index++;
                    }
                }
                }
            }
      
      
              // TLorentzVector definitions
              TLorentzVector Lepton_1  = TLorentzVector();
              TLorentzVector      MeT  = TLorentzVector();
      
    
                      // nominal values		      
              Lepton_1.SetPtEtaPhiE(lep_pt->at(goodlep_index), lep_eta->at(goodlep_index), lep_phi->at(goodlep_index),lep_E->at(goodlep_index));
              MeT.SetPtEtaPhiE(met_et, 0, met_phi , met_et);
      
              //Calculation of MTW
              float mtw = sqrt(2*Lepton_1.Pt()*MeT.Et()*(1-cos(Lepton_1.DeltaPhi(MeT))));

              // At least four good jets
              if(goodjet_n >= 4)
            {
              int goodjet1_index = goodjet_index[0]; // leading jet
              
              //At least two b-tagged jets
              if(goodbjet_n >= 2)
                {
                  int goodbjet1_index = goodbjet_index[0]; // leading b-jet
                  
                  // MTW > 30 GeV
                  if(mtw > 30000.)
                {

                  
                  // systematic variations on objects
  
                  // First, we sample a random number from a Gaussian distribution with a given mean and sigma
                  TRandom3* gRand = new TRandom3(0);
                  Double_t      mean_met = met_et; // nominal
                  Double_t      sigma_met = met_et_syst; // width corresponds to the uncertainty 
                  float met_et_variation = gRand->Gaus(mean_met,sigma_met); // this is the variation up and down
  
                  // leptons syst
                  Double_t      mean_lep = lep_pt->at(goodlep_index);
                  Double_t      sigma_lep = lep_pt_syst->at(goodlep_index) ; 
                  float lep_pt_variation = gRand->Gaus(mean_lep,sigma_lep);
                  
                  // lepton with syst 
                  TLorentzVector Lepton_1_syst  = TLorentzVector();
                  Lepton_1_syst.SetPtEtaPhiE(lep_pt_variation, lep_eta->at(goodlep_index), lep_phi->at(goodlep_index),lep_E->at(goodlep_index));

                  // MET with syst
                  TLorentzVector      MeT_syst  = TLorentzVector();
                  MeT_syst.SetPtEtaPhiE(met_et_variation, 0, met_phi , met_et_variation);

                  // MTW with syst variation on both MET and lepton pT
                  float mtw_syst = sqrt(2*Lepton_1_syst.Pt()*MeT_syst.Et()*(1-cos(Lepton_1_syst.DeltaPhi(MeT_syst))));
                  
                          // syst variation on jet pT
                  Double_t 	mean_jet = jet_pt->at(goodjet1_index)/1000.; 
                              Double_t 	sigma_jet = ( jet_pt_syst->at(goodjet1_index)/1000. ) ; 
                                  float leadjet_pt_variation = gRand->Gaus(mean_jet,sigma_jet); 
  
  
                  ///////// SAVE HISTOGRAMS /////////
                  double names_of_global_variable[]={met_et/1000.,mtw/1000.};

                  // correspond to the analysis cuts
                  if (MeT_syst.Et()/1000. > 30 && mtw_syst/1000. > 30) { 
                    FillHistogramsGlobal( MeT_syst.Et()/1000., weight, "hist_syst_etmiss");
                    FillHistogramsGlobal( mtw_syst/1000., weight, "hist_syst_mtw");
                  }
                  if (Lepton_1_syst.Pt()/1000. > 30) FillHistogramsLeadlept( Lepton_1_syst.Pt()/1000., weight, "hist_syst_leadleptpt");
  
  
                  double names_of_leadlep_variable[]={Lepton_1.Pt()/1000., Lepton_1.Eta(), Lepton_1.E()/1000., Lepton_1.Phi(), (double)lep_charge->at(goodlep_index), (double)lep_type->at(goodlep_index)};
  
                  double names_of_jet_variable[]={(double)goodjet_n, jet_pt->at(goodjet1_index)/1000.,jet_eta->at(goodjet1_index),(double)goodbjet_n, jet_pt->at(goodbjet1_index)/1000.,jet_eta->at(goodbjet1_index),leadjet_pt_variation};

                  TString histonames_of_global_variable[]={"hist_etmiss","hist_mtw"};
  
                  TString histonames_of_leadlep_variable[]={"hist_leadleptpt", "hist_leadlepteta","hist_leadleptE","hist_leadleptphi","hist_leadleptch","hist_leadleptID"};
  
                  TString histonames_of_jet_variable[]={"hist_n_jets","hist_leadjet_pt","hist_leadjet_eta","hist_n_bjets","hist_leadbjet_pt","hist_leadbjet_eta","hist_syst_leadjet_pt"};
  
                  int length_global = sizeof(names_of_global_variable)/sizeof(names_of_global_variable[0]);
  
                  int length_leadlep = sizeof(names_of_leadlep_variable)/sizeof(names_of_leadlep_variable[0]);
  
                  int length_leadjet = sizeof(names_of_jet_variable)/sizeof(names_of_jet_variable[0]);
  
                  for (int i=0; i<length_global; i++)
                    {
                      FillHistogramsGlobal( names_of_global_variable[i], weight, histonames_of_global_variable[i]);
                    }
                  
                  for (int i=0; i<length_leadlep; i++)
                    {
                      FillHistogramsLeadlept( names_of_leadlep_variable[i], weight, histonames_of_leadlep_variable[i]);
                    }
                  for (int i=0; i<length_leadjet; i++)
                    {
                      FillHistogramsLeadJet( names_of_jet_variable[i], weight, histonames_of_jet_variable[i]);
                    }
                  
  
                  // Invariant mass distribution of the 3-jets combination with the highest vector pT sum, a handle on the top mass
  
                  float PTjjjmax=0;
                  float Mjjjmax=0;
                  int a=0; int b=0; int c=0;
                                  
                      // iterate over 3 jets, build vectors
                  for ( int i = 0; i < goodjet_n; ++i) {
                    for ( int j = i + 1; j < goodjet_n; ++j) {
                      for ( int k = j + 1; k < goodjet_n; ++k) {
                    TLorentzVector jet1  = TLorentzVector(); jet1.SetPtEtaPhiE(jet_pt->at(goodjet_index[i]), jet_eta->at(goodjet_index[i]), jet_phi->at(goodjet_index[i]),jet_E->at(goodjet_index[i]));
                    TLorentzVector jet2  = TLorentzVector(); jet2.SetPtEtaPhiE(jet_pt->at(goodjet_index[j]), jet_eta->at(goodjet_index[j]), jet_phi->at(goodjet_index[j]),jet_E->at(goodjet_index[j]));
                    TLorentzVector jet3  = TLorentzVector(); jet3.SetPtEtaPhiE(jet_pt->at(goodjet_index[k]), jet_eta->at(goodjet_index[k]), jet_phi->at(goodjet_index[k]),jet_E->at(goodjet_index[k]));

                    // find largest pT of 3-jet system,
                    float PTjjjTemp = (jet1 + jet2 + jet3).Pt()/1000. ;

                    if (PTjjjTemp>PTjjjmax) { 
                      PTjjjmax=PTjjjTemp;  
                      Mjjjmax = (jet1 + jet2 + jet3).M()/1000.; // this is m(jjj) 
                      a=i; b=j; c=k; // store the indices
  
                      // among those jets, find largest pT of 2-jet system, a handle of the W-boson
                                          float PTjjTemp12 = (jet1 + jet2).Pt()/1000. ;
                                          float PTjjTemp13 = (jet1 + jet3).Pt()/1000. ;
                                          float PTjjTemp23 = (jet2 + jet3).Pt()/1000. ;
  
                                          if (PTjjTemp12 > PTjjTemp13 && PTjjTemp12 > PTjjTemp23) {a=i; b=j; c=k;}
                                          if (PTjjTemp13 > PTjjTemp12 && PTjjTemp13 > PTjjTemp23) {a=i; b=k; c=j;}
                                          if (PTjjTemp23 > PTjjTemp12 && PTjjTemp23 > PTjjTemp13) {a=j; b=k; c=i;}
  
                    }   
                      }
                    }
                  }
                  
                  // among the previous 3 jets, we search for those 2 that have the maximum pT, a handle on the W-boson mass
                  TLorentzVector j1  = TLorentzVector(); j1.SetPtEtaPhiE(jet_pt->at(goodjet_index[a]), jet_eta->at(goodjet_index[a]), jet_phi->at(goodjet_index[a]),jet_E->at(goodjet_index[a]));
                  TLorentzVector j2  = TLorentzVector(); j2.SetPtEtaPhiE(jet_pt->at(goodjet_index[b]), jet_eta->at(goodjet_index[b]), jet_phi->at(goodjet_index[b]),jet_E->at(goodjet_index[b]));
  
                  float Mjjmax= ( j1 + j2 ).M()/1000.; // first indices  
  
                  if ( Mjjjmax > 100 && Mjjjmax < 250)
                    FillHistogramsTTbar(Mjjjmax,weight,"hist_Topmass");
  
                  if ( Mjjmax > 50 && Mjjmax < 120)
                    FillHistogramsTTbar(Mjjmax,weight,"hist_Wmass");
  
  
                  // delete random number
                  gRand->Delete();

  
  
                }
      
                            }
            }
            }
        }
        }
    }
}

std::cout << "* Analysed a total of: " << nentries << " in this sample." << std::endl;

We will analyze, for example, the top mass histogram. We are selecting below a simple look for it.

In [ ]:
hist_Topmass->SetMarkerSize(2.0);
hist_Topmass->SetLineColor(kBlue);
hist_Topmass->SetFillColor(kBlue-10);

#### Final plot

In [ ]:
TCanvas *cz = new TCanvas("cz","cz",10,10,900,600);
TText tz; tz.SetTextFont(42); tz.SetTextAlign(21);
hist_Topmass->Draw();
cz->Draw();

#### Log Scale

In [ ]:
hist_Topmass->Draw("E3");
cz->SetLogy();
cz->Draw();

**Done!**